In [14]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

In [15]:
wine = load_wine()
X, y = wine.data, wine.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [16]:
class RBF(nn.Module):
    def __init__(self, input_dim, n_centers, output_dim, X_train):
        super(RBF, self).__init__()
        self.n_centers = n_centers


        kmeans = KMeans(n_clusters=n_centers, random_state=42)
        kmeans.fit(X_train.numpy())

        self.centers = nn.Parameter(torch.FloatTensor(kmeans.cluster_centers_))

        distances = torch.pdist(self.centers)
        self.sigma = nn.Parameter(torch.ones(n_centers) * distances.mean())

        self.fc = nn.Linear(n_centers, output_dim)

    def gaussian_rbf(self, x, center, sigma):
        """Gaussian activation function"""
        x_expanded = x.unsqueeze(1)
        c_expanded = center.unsqueeze(0)
        dist = torch.sum((x_expanded - c_expanded) ** 2, dim=-1)  # [batch_size, n_centers]
        return torch.exp(-dist / (2 * sigma ** 2))

    def forward(self, x):
        rbf = self.gaussian_rbf(x, self.centers, self.sigma)
        return self.fc(rbf)

In [37]:
num_classes = len(np.unique(y))
num_centers = 10
input_dim = X.shape[1]

rbf = RBF(input_dim, num_centers, num_classes, X_train)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rbf.parameters(), lr=0.01)

In [38]:
def train_rbf(model, X_train, y_train, X_test, y_test, n_epochs=100):
    for epoch in range(n_epochs):

        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()


        model.eval()
        with torch.no_grad():
            test_op = model(X_test)
            _, pred = torch.max(test_op, 1)
            acc = (pred == y_test).sum().item() / y_test.size(0)
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")

In [41]:
train_rbf(rbf, X_train, y_train, X_test, y_test, n_epochs=10)

Epoch 1/10, Loss: 0.9331, Accuracy: 0.6111
Epoch 2/10, Loss: 0.9230, Accuracy: 0.6111
Epoch 3/10, Loss: 0.9129, Accuracy: 0.6389
Epoch 4/10, Loss: 0.9028, Accuracy: 0.6389
Epoch 5/10, Loss: 0.8927, Accuracy: 0.6667
Epoch 6/10, Loss: 0.8825, Accuracy: 0.6944
Epoch 7/10, Loss: 0.8724, Accuracy: 0.7222
Epoch 8/10, Loss: 0.8622, Accuracy: 0.8056
Epoch 9/10, Loss: 0.8520, Accuracy: 0.8333
Epoch 10/10, Loss: 0.8418, Accuracy: 0.8889


In [49]:
def plot(model, X, y):
    # Convert PyTorch tensors to NumPy arrays
    X_np = X.numpy()
    y_np = y.numpy()

    # Define grid boundaries with some padding
    X_min, X_max = X_np[:,0].min() - 1, X_np[:,0].max() + 1
    y_min, y_max = X_np[:,1].min() - 1, X_np[:,1].max() + 1  # Changed to X_np[:,1] for y-axis

    # Create meshgrid for plotting decision boundary
    xx, yy = np.meshgrid(np.arange(X_min, X_max, 100),
                         np.arange(y_min, y_max, 100))

    # Create grid points and convert to PyTorch tensor
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

    # Get model predictions
    with torch.no_grad():
        Z = model(grid)
        Z = torch.argmax(Z, axis=1).reshape(xx.shape)

    # Plot decision boundary and data points
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X_np[:, 0], X_np[:, 1], c=y_np, edgecolors='k')
    plt.show()

In [53]:
plot(rbf,X_train,y_train)

RuntimeError: The size of tensor a (2) must match the size of tensor b (13) at non-singleton dimension 2